# NIRSpec IFU Optimal Point Source Extraction

## Imports 

* _numpy_ for array math
* _scipy_ for ndcube gaussian smoothing
* _specutils_ for Spectrum1D data model and cube manipulation
* _jdaviz_ : Cubeviz data visualization tool
* _photutils_ to define circular apertures
* _astropy.io_ for reading and writing FITS cubes and images
* _astropy.wcs, units, coordinates_ for defining and reading WCS
* _astropy.stats_ for sigma_clipping
* _astropy.utils_ for downloading files from URLs
* _matplotlib_ for plotting spectra and images

In [ ]:
#Resize notebook to full width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import scipy

import specutils
from specutils import Spectrum1D, SpectralRegion
from specutils.manipulation import extract_region, spectral_slab

from jdaviz import CubeViz

from photutils import CircularAperture, SkyCircularAperture, aperture_photometry
from photutils.detection import DAOStarFinder

from regions import PixCoord, CirclePixelRegion

from astropy.io import fits
from astropy.stats import sigma_clip
from astropy.stats import sigma_clipped_stats
from astropy.utils.data import download_file
import astropy.units as u

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
#%pip install jwst

## Introduction

This notebook illustrates various extraction methods for a point source in JWST NIRSpec IFU data. First we
demonstrate a number of regular extraction techniques, including subset extraction with Cubeviz, simple sum over spaxels, cylindrical aperture, and conical aperture photometry. Then we compare optimal extraction using a WebbPSF model PSF to optimal extraction using a reference star PSF. 


## Read in Simulated NIRSpec IFU Cube

A faint (quasar) point source was simulated using the NIRSpec Instrument Performance Simulator (IPS), then run through the JWST Spec2 pipeline. We will use this for our science dataset.

Read in the data both with fits.open (to inspect the data structure and header) and Spectrum1D.read to utilize specutils functionality.

In [ ]:
# NIRSpec IFU science data cube
BoxPath = "https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/IFU_optimal_extraction/"
fname = "NRS00001-faintQSO-F100LP-G140H-01_1_491_SE_2020-08-25T12h15m00_s3d.fits"
filename = BoxPath + fname

# Open and inspect the file with astropy.fits.open
with fits.open(fname, memmap=False) as hdulist:
    hdulist.info()
    sci = hdulist['SCI'].data
    err = hdulist['ERR'].data  
    
# Load original (untrimmed version) with Spectrum1D    
spec1d_untrimmed = Spectrum1D.read(fname, format='JWST s3d')
wavelength_untrimmed = spec1d_untrimmed.spectral_axis
print()
print("Wavelength: ", wavelength_untrimmed)

*Developer Notes:* 

(1) Can we fix or suppress the AsdfWarning? It goes away if 'jwst' package is installed.

(2) The default loader doesn't read in the uncertainty.

(3) Unable to print the Spectrum1D object.

(4) Spectrum1D 'JWST s3d' has trouble with the Box URL, but works with a local filename.

## Visualize Science Data with Cubeviz

In [ ]:
cubeviz = CubeViz()
cubeviz.app

### UI Instructions:
* Load science datacube into Cubeviz using the next code cell below
* Go to the Hammer-and-Screwdriver icon: Gear icon: Layer in the leftmost image viewer 
* In that tab, change the Linear stretch to 90 percentile to see the faint QSO target at (x,y) ~ (17, 21)
* Scrubbing through the cube also helps to locate the source
* Select a circular subset region centered on the source. 
* Note that the region is pixelated and doesn't include fractional pixels
* Change the collapse method to "Sum" in spectrum viewer: Gear icon : Viewer 
* --This "Sum" method yields our subset extraction
* Change the vertical zoom to see the spectral features in the Subset spectrum

*Developer Notes*: 

(5) Image viewer contrast settings change when you click on the side bar to expand/contract jupyter scroll window

(6) Spectrum viewer: Viewer cube collapse method should default to Sum (not Maximum)

(7) Spectrum viewer y scale returns to autoscale when the region is moved, and y-zoom has to be adjusted again


## Load Cube into Cubeviz

In [ ]:
# Data from local directory
cubeviz.app.load_data(fname)

# Data from url:
#df = download_file(filename)
#cubeviz.app.load_data(df)

*Developer Note:* 

(8) Spectral cube gives "No spectral axis found" warning for each extension of JWST NIRSpec IFU datacubes.

## Export Region from Cubeviz
Export the region defined by the user in Cubeviz as an astropy CirclePixel Region, which has units of pixels.

In [ ]:
cubeviz_data = cubeviz.app.data_collection[0]
try:
    region1 = cubeviz.app.get_subsets_from_viewer('spectrum-viewer')['Subset 1']
    print(region1)
    region1_exists = True
    center1_xy = [region1.center.x, region1.center.y]  
    r_pix = region1.radius

except Exception:
    print("There are no regions selected in the cube viewer.")
    region1_exists = False
    center1_xy = [17.1, 20.]
    r_pix = 6.0
      

*Developer Note:*

(9) Resolve this glue core "Future Warning" about multidemensional indexing.

## Extract Subset Spectrum in Cubeviz Spectrum Viewer
Retrieve the spectrum (Subset1) of the user-defined region from the Spectrum Viewer as a Spectrum1D object. Trim to remove bad wavelength ranges.

In [ ]:
trim_region = SpectralRegion(1.0*u.um, 1.43*u.um)
try:
    spectrum_subset1_untrimmed = cubeviz.app.get_data_from_viewer('spectrum-viewer')['Subset 1']
    print(spectrum_subset1_untrimmed)
    
    # Trim the extracted spectrum
    spectrum_subset1 = extract_region(spectrum_subset1_untrimmed, trim_region)
    
    print()
    print('Wavelength:', spectrum_subset1_untrimmed.spectral_axis)
    print('Trimmed:', spectrum_subset1.spectral_axis)
    print()

except Exception:
    print("There are no subsets selected in the spectrum viewer.")
    

#spectrum_subset1_untrimmed = spectrum_subset1_untrimmed.with_spectral_unit(u.um)


*Developer Notes:* 

(10) Why does Spectrum1D.with_spectral_unit not work on the dataset from Subset 1?

(11) Why does extract_region complain that "No observer defined on WCS"?

## Extract Spectrum by Sum Over Spaxels

First trim the cube to remove bad edges and wavelengths. Perform a simple numpy sum over all spaxels in the cube as a rudimentary extraction method. Also sum over wavelength to collapse the cube.  

In [ ]:
# Trim the data cube and adjust region location
wave_trim =[1.0*u.um,1.43*u.um]
x_trim = [2,-1]
y_trim = [5, -4]
spec1d = spectral_slab(spec1d_untrimmed, wave_trim[0], wave_trim[1])[x_trim[0]:x_trim[1],y_trim[0]:y_trim[1],:]
wavelength = spec1d.spectral_axis

#Adjust region location in trimmed cube
center1_trim = PixCoord(x=center1_xy[0]-x_trim[0], y=center1_xy[1]-y_trim[0])
region1_trim = CirclePixelRegion(center=center1_trim, radius=r_pix)
print(region1_trim)

# Untrimmed vs. Trimmed cube dimensions 
print('')
print('Untrimmed:')
print('Shape:',np.shape(spec1d_untrimmed))
print('Wavelength:', wavelength_untrimmed)
print()
print('Trimmed:')
print('Shape:', np.shape(spec1d))
print('Wavelength:', wavelength)

# Sum over spaxels
fnu_sum = np.sum(spec1d.flux, axis=(0, 1))
print('Flux:', fnu_sum)

#Sum over wavelength
cube_sum = np.sum(spec1d.flux.value, axis=2)

# Plots
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5)) 
ax1.plot(wavelength, fnu_sum)
ax1.set_xlim(wavelength_untrimmed.value[0], wavelength_untrimmed.value[-1])
ax1.set_title("Spaxel sums")
ax1.set_xlabel("Wavelength (um)")  
ax1.set_ylabel("SCA491 Flux Density")

ax2.imshow(np.transpose(cube_sum), norm=LogNorm())
patch = region1.as_artist(facecolor='none', edgecolor='red', lw=2)
patch = region1_trim.as_artist(facecolor='none', edgecolor='red', lw=2)
ax2.add_patch(patch)
ax2.set_title("Slice sums")

plt.show()

Left: Sum over good spaxels.  Right: Sum over good wavelengths, with Cubeviz circular extraction region overplotted. Origin at upper left corner.

*Developer Notes:* 

(12) The cube (x,y) coordinates are transposed in the Spectrum1D object with respect to the astropy fits cube and the Cubeviz image display, so we have to transpose the image when displaying.

(13) Would be nice to have specutils function(s) or Spectrum1D method(s) that do the equivalent of the Cubeviz collapse plugin, preserving the units and uncertainties, like this: cube_sum = spec1d.collapse(axis=2)

## Extract Spectrum in Constant Radius Circular Aperture (Cylinder)
This method is appropriate for an extended source.

In [ ]:
#Use photutils.aperture_photometry to measure flux in constant aperture
aperture = CircularAperture((center1_trim.x,center1_trim.y),r=r_pix)
print(aperture)
spec1d_len = len(wavelength)
cylinder_sum = []
for idx in range(spec1d_len):
    phot_table = aperture_photometry(spec1d.flux.value[:, :, idx], aperture)
    cylinder_sum.append(phot_table['aperture_sum'][0])
cylinder = Spectrum1D(flux=np.array(cylinder_sum)*u.MJy/u.sr, spectral_axis=spec1d.spectral_axis)
print(cylinder)

*Developer Note:*  Is there a way to retrieve the coordinates (RA, Dec) of the Subset1 region, for use in a SkyCircularAperture?

## Extract Spectrum in Linearly Expanding Circular Aperture (Cone)
This method is appropriate for a point source PSF with width proportional to wavelength

In [ ]:
#Use photutils.aperture_photometry to measure flux in expanding aperture
cone_sum = []
for idx in range(spec1d_len):
    r_cone = r_pix * wavelength.value[idx]/ wavelength.value[0]
    aperture_cone = CircularAperture((center1_trim.x,center1_trim.y), r=r_cone)
    phot_table = aperture_photometry(spec1d.flux.value[:, :, idx], aperture_cone)
    cone_sum.append(phot_table['aperture_sum'][0])
    
cone = Spectrum1D(flux=np.array(cone_sum)*u.MJy/u.sr, spectral_axis=spec1d.spectral_axis)
print(cone)


## Plot and Compare Non-optimal Spectral Extractions
Compare spectra extracted in cylinder, cone, Cubeviz subset.

In [ ]:
print(spectrum_subset1)
f, (ax1) = plt.subplots(1, 1, figsize=(15, 5)) 

ax1.set_title("Non-optimal spectral extractions")
ax1.set_xlabel("Observed Wavelength (microns)")  
ax1.set_ylabel("Flux Density")
ax1.set_xlim(0.99, 1.442)
ax1.set_ylim(0, 0.6)

print(cylinder.spectral_axis)
ax1.plot(wavelength, cylinder.flux.value, label="Cylinder", c='b')
ax1.plot(wavelength, cone.flux.value, label="Cone", c='darkorange', alpha=0.5)
try:
    ax1.plot(wavelength, spectrum_subset1.flux.value, c='r', label="Subset1", alpha=0.4)
except Exception:
    print("There is no Cubeviz Subset1 spectrum to plot.")
ax1.legend()

plt.show()

Comparison of the (non-optimal) cylindrical, conical, and Cubeviz subset spectral extractions. 
The conical extraction captures slightly more flux but is noisier than the other spectra at long wavelengths.
Red-shifted Broad H-beta and narrow [O III] lines  are visible in the quasar spectra. 

## WebbPSF  Model PSF for Optimal Extraction
Generate PSF model cube using WebbPSF for NIRSpec IFU, or read in precomputed PSF model cube.

Caution! The WebbPSF model takes about 10 hr to run.  Uncomment the following cell to do so. Otherwise, read in the precomputed WebbPSF model, which covers the full F100LP/G140H wavelength range (blue and red). For other filter/grating combinations, uncomment and run the cell below using the wavelengths from the science data set.

In [ ]:
'''
#WebbPSF imports
%pylab inline
import webbpsf

#WebbPSF commands used to create PSF model cube
ns.image_mask = "IFU"  # Sets to 3x3 arcsec square mask
ns = webbpsf.NIRSpec()
wavelengths = wavelength*1.0E-6
psfcube = ns.calc_datacube(wavelengths, fov_pixels=30, oversample=4,  add_distortion=True)
psfcube.writeto("Webbpsf_ifucube.fits")
'''

In [ ]:
BoxPath = "https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/IFU_optimal_extraction/"
psf_fname = "Webbpsf_ifucube.fits"
psf_filename = BoxPath + psf_fname


# Load with astropy.fits.open
with fits.open(psf_fname, memmap=False) as hdulist:
    psf_model = hdulist['DET_SAMP'].data
    psf_hdr = hdulist['DET_SAMP'].header
    hdulist.info()    

#WebbPSF model wavelengths
wave0_webbpsf = psf_hdr['WAVE0']*1.0E6
dlam_webbpsf = (psf_hdr['WVLN0001']-psf_hdr['WVLN0000'])*1.0E6 #0.000235
nwave_webbpsf = psf_hdr['NAXIS3']
wavelength_webbpsf_untrimmed = np.array(wave0_webbpsf + np.arange(nwave_webbpsf) * dlam_webbpsf)

#Trim wavelengths
#Note that the trimmed wavelengths of the WebbPSF model are slightly different from the science data
#Also need to extend them by 1 bin to match length of science data wavelength array
good_webbpsf_wave = np.where((wavelength_webbpsf_untrimmed >= wave_trim[0].value) & (wavelength_webbpsf_untrimmed <= wave_trim[1].value))[0]
good_webbpsf_wave = np.append(good_webbpsf_wave, good_webbpsf_wave[-1]+1) 
wavelength_webbpsf = wavelength_webbpsf_untrimmed[good_webbpsf_wave] * u.um
print("Data Wavelength: ", wavelength)
print("WebbPSF Wavelength", wavelength_webbpsf)

#Trim datacube
psf_model=psf_model[good_webbpsf_wave[0]:good_webbpsf_wave[-1]+1,:,:]
print(np.shape(psf_model))

# Sum over wavelength
psf_model_sum = np.sum(psf_model, axis=0)

# Sum over spaxels
psf_model_fnusum = np.sum(psf_model, axis=(1, 2))


*Developer Note:*  The file Webbpsf_ifucube.fits is large (946.3 MB) and takes some time to load from Box.
It might behoove the user to download it to a local directory and retrieve it from there.

## Align Model PSF Cube with Science Data
Window the simulated data. Flip, smooth, and shift the model PSF cube to align with the data. 

In [ ]:
#Window the science data
good_wave = np.where((wavelength_untrimmed >= wave_trim[0]) & (wavelength_untrimmed<=wave_trim[1]))[0]
data_win = np.nan_to_num(np.array(sci)[good_wave[0]:good_wave[-1]+1, 5:-4, 3:])
data_var = np.nan_to_num(np.array(err)[good_wave[0]:good_wave[-1]+1, 5:-4, 3:]) 
data_sum = np.sum(data_win, axis=0)
print(np.shape(sci), np.shape(data_win), np.shape(data_var))

# Flip model PSF left-right.  For some unknown reason, WebbPSF is flipped with respect to the IPS simulation.
#psf_model_fliplr = psf_model
psf_model_fliplr = psf_model[:, ::-1, :]

# Smooth model to match data smoothing built into cube_build algorithm
# EMSM smoothing for G140H grating
scalerad = 0.046  # sigma (arcsec)
pixelscale = 0.1
scalerad_pix = scalerad / pixelscale
psf_model_smoothed = scipy.ndimage.filters.gaussian_filter(psf_model_fliplr, (0.0, scalerad_pix, scalerad_pix), 
                                                           order=0, mode='reflect', cval=0.0, truncate=10.0)
# Find location of star in collapsed data 
data_mean, data_median, data_std = sigma_clipped_stats(data_sum, sigma=3.0) 
daofind = DAOStarFinder(fwhm=3.0, threshold=5.*data_std)
data_sources = daofind(data_sum-data_median) 
for col in data_sources.colnames:  
    data_sources[col].info.format = '%.6g'  
print("Sources detected in data:")
print(data_sources)  
print()

# Find location of star in model
model_mean, model_median, model_std = sigma_clipped_stats(psf_model_sum, sigma=3.0) 
daofind = DAOStarFinder(fwhm=3.0, threshold=5.*model_std)
model_sources = daofind(psf_model_sum-model_median) 
for col in model_sources.colnames:  
    model_sources[col].info.format = '%.6g'  
print("Sources detected in model:")
print(model_sources)  
print()

# (x,y) shift between model and data
shiftx = data_sources['xcentroid'].data[0] - model_sources['xcentroid'].data[0]
shifty = data_sources['ycentroid'].data[0] - model_sources['ycentroid'].data[0]
print("Shift (x,y) = ", shiftx, shifty)
print()

# Shift model PSF using linear interpolation
psf_model_aligned = scipy.ndimage.shift(psf_model_smoothed, (0.0, shifty, shiftx), order=1, 
                                        mode='constant', cval=0.0, prefilter=True)
# Replace NaNs
profile = np.nan_to_num(psf_model_aligned) 

# Sum over wavelength
psf_model_aligned_sum = np.sum(psf_model_aligned, axis=0)

# Scale factor for PSF subtraction
psf_sum_min = np.amin(psf_model_aligned_sum)
psf_sum_max = np.amax(psf_model_aligned_sum)
scalefactor = np.amax(cube_sum) / psf_sum_max

# Plots
f, ([ax1, ax2, ax3], [ax4, ax5, ax6]) = plt.subplots(2, 3, figsize=(10, 10)) 

ax1.set_title("PSF slice sum")
ax1.imshow(psf_model_aligned_sum, norm=LogNorm())

ax2.set_title("Science Data slice sum")
ax2.imshow(data_sum, norm=LogNorm()) 

ax3.set_title("Data / PSF Ratio")
ax3.imshow(data_sum / psf_model_aligned_sum, norm=LogNorm())

ax4.set_title("PSF Model integrated flux")
ax4.plot(psf_model_fnusum)

ax5.set_title("Data - PSF")
ax5.imshow(data_sum - scalefactor * psf_model_aligned_sum)

im6 = ax6.imshow(np.log10(np.absolute(data_sum - scalefactor * psf_model_aligned_sum)))
plt.colorbar(im6)
ax6.set_title("log abs(Data - PSF)")

plt.show()

_Figure top row_: Comparison of smoothed, aligned WebbPSF (left) to IPS simulation (center). 

_Figure bottom row_: Integrated WebbPSF model flux (left) decreases with wavelength as PSF expands outside of the FOV. 
Differences (center, right) between the model PSF and IPS-simulated PSF.

## Optimal Extraction using WebbPSF Model
Optimal extraction (Horne 1986, PASP, 98, 609) weights the flux contributions to a spectrum by their signal-to-noise ratio (SNR). Dividing the simulated data by the model PSF gives an estimate of the total flux density spectrum in each spaxel. A weighted average of these estimates over all spaxels yields the optimally extracted spectrum over the cube. In the faint source limit, where the noise is background-dominated, optimal extraction inside a 3-sigma radius can increase the effective exposure time by a factor of 1.69 (Horne et al. 1986). In the bright source limit, where the noise is dominated by the Poisson statistics of the source, optimal extraction is formally identical to a straight sum over spaxels for a perfect PSF model. 

We use the WebbPSF PSF model for this first attempt at optimal extraction.

In [ ]:
# Optimal extraction, using model profile weight and variance cube from the simulated data
optimal_weight = profile ** 2 / data_var
optimal_weight_norm = np.sum(optimal_weight, axis=(1, 2))
spectrum_optimal = np.sum(profile * data_win / data_var, axis=(1, 2)) / optimal_weight_norm

#Scale may be off if PSF doesn't match perfectly
opt_scalefactor = np.median(np.nan_to_num(cone_sum / spectrum_optimal))  
print("Scale Factor:", opt_scalefactor)

# Trim the extracted spectrum
spec1d_optimal = Spectrum1D(flux=spectrum_optimal*u.MJy/u.sr, spectral_axis=wavelength)

print(spec1d_optimal)

# Plots
f, (ax1) = plt.subplots(1, 1, figsize=(12, 6)) 
ax1.set_title("Optimal Extraction Comparison")
ax1.set_xlabel("Observed Wavelength (microns)") 
ax1.set_ylabel("Flux Density")
ax1.set_ylim(0, 0.5)
ax1.plot(wavelength, cone.flux.value, label="Conical Extraction", alpha=0.5)
ax1.plot(wavelength, spec1d_optimal.flux.value, label="Optimal")
ax1.legend()

plt.show()

The optimally extracted spectrum is less noisy than the aperture extraction.

## Optimal Extraction with (Simulated) Reference Star PSF
A real (or simulated in this case) IFU observation of a star may be used for the PSF model rather than WebbPSF.  We employ a NIRSpec IPS simulated PSF, which matches our data better than the WebbPSF model.  We don't have to shift or smooth the PSF model because it was simulated at the same dither/detector position as the data. When using a real observation of a star for the PSF model, make sure it was observed at the same dither positions. It is also beneficial to reduce and extract both simulated datasets in the 'ifualign' detector coordinate system, so that we don't have to rotate the PSF star to match the science data.

In [ ]:
BoxPath = "https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/IFU_optimal_extraction/"
fname_star = "NRS00001-brightQSO-F100LP-G140H-01_1_491_SE_2020-08-26T12h15m00_s3d.fits"
filename_star = BoxPath + fname_star

# Open and inspect the file
with fits.open(fname_star, memmap=False) as hdulist:
    sci_star = hdulist['SCI'].data
    hdulist.info()
    
#Window the reference star data
star_win = np.nan_to_num(np.array(sci_star)[good_wave[0]:good_wave[-1]+1, 5:-4, 3:])
print(np.shape(star_win))

#Sum over wavelength
star_sum = np.sum(star_win, axis=0)

# Reference star spectrum
#star_fnusum = np.sum(star_win, axis=(1, 2))

# Normalize PSF star profile to unity. (The flux will still be slightly off. Please see Developer's Note below.)
star_spectrum = []
star_norm = []
for idx in range(len(wavelength)):
    r_cone = r_pix * wavelength.value[idx] / wavelength.value[0]
    aperture_cone = CircularAperture((center1_trim.x,center1_trim.y), r=r_cone)
    phot_table = aperture_photometry(star_win[idx, :, :], aperture_cone)
    star_phot = phot_table['aperture_sum'][0]
    star_spectrum.append(star_phot)
    star_norm.append(star_win[idx,:,:] / star_phot)
    
star_spec1d = Spectrum1D(flux=np.array(star_spectrum)*u.MJy/u.sr, spectral_axis=wavelength)
print(star_spec1d)  
    
profile_star = np.array(star_norm)
print(np.shape(profile_star))
    
# Sum over wavelength
profile_star_sum = np.nan_to_num(np.sum(profile_star, axis=0))
print(np.shape(profile_star_sum))

# Scale factor for PSF subtraction
profile_star_sum_max = np.amax(profile_star_sum)
star_scalefactor = np.amax(data_sum) / profile_star_sum_max
print("Data/Star scale factor: ", star_scalefactor)

# Plots
f, ([ax1, ax2, ax3], [ax4, ax5, ax6]) = plt.subplots(2, 3, figsize=(10, 10)) 

ax1.imshow(profile_star_sum, norm=LogNorm())
ax1.set_title("PSF Star Slice sum")

ax2.imshow(data_sum, norm=LogNorm()) 
ax2.set_title("Science Data Slice sum")

ax3.imshow(np.abs(data_sum / profile_star_sum),norm=LogNorm())
ax3.set_title("Data/Star_PSF Ratio")

star_model_ratio = profile_star_sum / psf_model_sum
ax4.imshow(star_model_ratio, norm=LogNorm())
ax4.set_title("Star PSF/WebbPSF")

ax5.imshow(data_sum - star_scalefactor * profile_star_sum)
ax5.set_title("Data - Star PSF")

ax6.imshow(np.log10(np.absolute(data_sum - star_scalefactor * profile_star_sum)))
ax6.set_title("log abs(Data - Star PSF)")

plt.show()

_Figure top row_: Comparison of PSF star and science data. Bottom left: ratio of PSF star to WebbPSF model shows 
significant differences that can affect the quality of the optimal extraction.  Bottom right:
Difference of PSF star from science data shows they are well matched, with a scale factor of 0.175. 

*Developer Note:* It would be good to renormalize the PSF profile to account for the fraction of flux lost outside of the detector. Otherwise the extracted flux will be low by a factor of roughly 0.972 to 0.980.

In [ ]:
# Optimal extraction, using model profile weight and variance cube from the simulated data
optimal_weight = profile_star**2 / data_var
optimal_weight_norm = np.sum(optimal_weight, axis=(1, 2))
spectrum_optimal_star = np.sum(profile_star * data_win / data_var, axis=(1, 2)) / optimal_weight_norm

# Create Spectrum1D object
spec1d_optimal_star = Spectrum1D(flux=spectrum_optimal_star*u.MJy/u.sr, spectral_axis=wavelength)
print(spec1d_optimal_star)

# Plots
f, (ax1) = plt.subplots(1, 1, figsize=(12, 6)) 
ax1.set_title("Optimal Extraction Comparison")
ax1.set_xlabel("Observed Wavelength (microns)") 
ax1.set_ylabel("Flux Density")
ax1.set_ylim(0, 0.5)

ax1.plot(wavelength, cone.flux.value, label="Conical Extraction", alpha=0.5)
ax1.plot(wavelength, spec1d_optimal.flux.value, label="Optimal with WebbPSF model", alpha=0.5)
ax1.plot(wavelength, spec1d_optimal_star.flux.value, label="Optimal with ref. star")
ax1.legend()

plt.show()

The optimal extraction with the perfectly matched PSF star is less noisy than that achieved with WebbPSF.  

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>

Notebook created by Patrick Ogle and James Davies.